In [1]:
#! pip install sklearn

In [2]:
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
from data.load_data import load_trustpilot_data
from data.text_pre import clean_text

In [3]:
df_trust = load_trustpilot_data()
df_trust = clean_text(df_trust, 'text')

In [4]:
df = pd.DataFrame({'label':df_trust.y, 'text':df_trust.text})

In [5]:
from sklearn.model_selection import train_test_split
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [6]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [7]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [34]:
itos='itos'
lm_path='lm_best_with_opt' 
learn = language_model_learner(data_lm, pretrained_fnames=[lm_path,itos], drop_mult=0.7, arch=AWD_LSTM)

In [ ]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.save_encoder('sentiment_enc')

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.7, arch=AWD_LSTM)
learn.load_encoder('sentiment_enc')

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)

tensor(0)


In [ ]:
pd.crosstab(predictions, targets)

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report
print(roc_auc_score(targets, predictions))
print(classification_report(targets, predictions))